In [1]:
import pandas as pd
import re

# Paso 1: Leer los logs y extraer los prompts
def extract_prompts_from_logs(log_file):
    prompts = {}  # Diccionario para almacenar {fila: prompt}
    pattern = r"Prompt generado para fila (\d+): (.+)"  # Patrón para extraer fila y prompt

    with open(log_file, "r", encoding="utf-8") as file:
        for line in file:
            match = re.search(pattern, line)
            if match:
                row_number = int(match.group(1))  # Número de fila
                prompt = match.group(2).strip()  # Prompt
                prompts[row_number] = prompt  # Almacenar en el diccionario

    return prompts

# Paso 2: Leer el CSV y agregar la columna "prompt"
def update_csv_with_prompts(csv_path, log_file, output_path):
    # Leer el CSV original
    df = pd.read_csv(csv_path)

    # Extraer los prompts del log
    prompts = extract_prompts_from_logs(log_file)

    # Inicializar la columna "prompt" con valores vacíos
    df['prompt'] = ""  # Añadir columna vacía

    # Asignar los prompts a las filas correspondientes
    for row_number, prompt in prompts.items():
        if row_number < len(df):  # Asegurarse de que la fila exista en el CSV
            df.at[row_number, 'prompt'] = prompt

    # Guardar el CSV actualizado
    df.to_csv(output_path, index=False)
    print(f"Archivo actualizado guardado en: {output_path}")

# Configuración de rutas
log_file = "togetherDiego.log"  # Ruta del archivo de logs
csv_path = "parte_1.csv"  # Ruta del archivo CSV original
output_path = "parte_1_con_prompts.csv"  # Ruta del archivo CSV actualizado

# Ejecutar el proceso
update_csv_with_prompts(csv_path, log_file, output_path)

Archivo actualizado guardado en: parte_1_con_prompts.csv


De los logs, tomar los prompts y ponerlos en un csv donde haga matcheo con `file_name`. Nos devuelve: el csv con los prompts actualizado, el csv con las filas sin que tengan prompts
> EL csv entrante, TIENE la columna `prompt` o `generated_prompt`

In [5]:
import pandas as pd
import re
import os

def extraer_prompts_del_log(archivo_log):

    prompts_extraidos = {}
    patron = r'Prompt generado para fila ([^:]+): "(.*?)"'
    
    with open(archivo_log, 'r', encoding='utf-8') as f:
        contenido = f.read()
    
    matches = re.findall(patron, contenido)
    for file_name, prompt in matches:
        prompts_extraidos[file_name.strip()] = prompt.strip()
    
    print(f"Se encontraron {len(prompts_extraidos)} prompts en el archivo de log.")
    return prompts_extraidos

def actualizar_csv_con_prompts(archivo_csv, prompts_dict, guardar_como):
    """
    Actualiza el CSV original con los prompts extraídos del log.
    """
    # Leer el CSV original
    df = pd.read_csv(archivo_csv)
    
    # Contador de prompts añadidos
    prompts_anadidos = 0
    
    # Actualizar los prompts faltantes
    for idx, row in df.iterrows():
        file_name = row['file_name']
        
        # Si la fila no tiene prompt y existe en nuestro diccionario de prompts
        if pd.isna(row['prompt']) and file_name in prompts_dict:
            df.at[idx, 'prompt'] = prompts_dict[file_name]
            prompts_anadidos += 1
    
    # Guardar el CSV actualizado
    df.to_csv(guardar_como, index=False)
    print(f"Se añadieron {prompts_anadidos} prompts al CSV. Guardado como {guardar_como}")
    
    return df

def crear_csv_filas_sin_prompt(df, guardar_como):
    """
    Crea un nuevo CSV solamente con las filas que aún no tienen prompt.
    """
    # Filtrar las filas sin prompt
    df_sin_prompt = df[df['prompt'].isna()]
    
    # Guardar el nuevo CSV
    df_sin_prompt.to_csv(guardar_como, index=False)
    print(f"Se creó un CSV con {len(df_sin_prompt)} filas sin prompt. Guardado como {guardar_como}")

def main():
    # Definir rutas de archivos
    archivo_csv = 'parte_4_con_prompts.csv'
    archivo_log = 'merge34.log'
    csv_actualizado = 'parte_4_completo.csv'
    csv_filas_sin_prompt = 'parte_4_mergefaltantes.csv'
    
    # Comprobar que los archivos existen
    for archivo in [archivo_csv, archivo_log]:
        if not os.path.exists(archivo):
            print(f"Error: El archivo {archivo} no existe.")
            return
    
    # Extraer prompts del archivo de log
    prompts = extraer_prompts_del_log(archivo_log)
    
    # Actualizar el CSV con los prompts extraídos
    df_actualizado = actualizar_csv_con_prompts(archivo_csv, prompts, csv_actualizado)
    columnas_con_faltantes = df_actualizado.columns[df_actualizado.isnull().any()].tolist()

    if columnas_con_faltantes:
        print("Las siguientes columnas tienen valores faltantes:")
        for columna in columnas_con_faltantes:
            cantidad_faltantes = df_actualizado[columna].isnull().sum()
            print(f"- Columna '{columna}' tiene {cantidad_faltantes} valores faltantes.")
    else:
        print("No hay columnas con valores faltantes.")
    
    # Crear un CSV con las filas que aún no tienen prompt
    crear_csv_filas_sin_prompt(df_actualizado, csv_filas_sin_prompt)
    
    print("Proceso completado con éxito.")

if __name__ == "__main__":
    main()
    
    

Se encontraron 9605 prompts en el archivo de log.
Se añadieron 5260 prompts al CSV. Guardado como parte_4_completo.csv
Las siguientes columnas tienen valores faltantes:
- Columna 'painting_name' tiene 2 valores faltantes.
- Columna 'phash' tiene 543 valores faltantes.
Se creó un CSV con 0 filas sin prompt. Guardado como parte_4_mergefaltantes.csv
Proceso completado con éxito.


De los logs, toma los prompts y ponerlos en un csv donde haga matcheo con `file_name` !Important. Nos devuelve: el csv con los prompts actualizado, el csv con las filas sin que tengan prompts

> EL csv entrante, NO tiene la columna `prompt` o `generated_prompt`

In [2]:
import pandas as pd
import re
import os

def extraer_prompts_del_log(archivo_log):
    """
    Extrae los prompts del archivo de log.
    Retorna un diccionario donde la clave es el nombre del archivo y el valor es el prompt.
    """
    prompts_extraidos = {}
    patron = r'Prompt generado para fila ([^:]+): "(.*?)"'
    
    with open(archivo_log, 'r', encoding='utf-8') as f:
        contenido = f.read()
    
    matches = re.findall(patron, contenido)
    for file_name, prompt in matches:
        prompts_extraidos[file_name.strip()] = prompt.strip()
    
    print(f"Se encontraron {len(prompts_extraidos)} prompts en el archivo de log.")
    return prompts_extraidos

def actualizar_csv_con_prompts(archivo_csv, prompts_dict, guardar_como):
    """
    Actualiza el CSV original con los prompts extraídos del log.
    Si no existe la columna 'prompt', la añade.
    """
    # Leer el CSV original
    df = pd.read_csv(archivo_csv)
    
    # Verificar si existe la columna 'prompt' y crearla si no existe
    if 'generated_prompt' not in df.columns:
        df['generated_prompt'] = None
        print("Se ha añadido la columna 'generated_prompt' que no existía en el CSV original.")
    
    # Contador de prompts añadidos
    prompts_anadidos = 0
    
    # Actualizar los prompts faltantes
    for idx, row in df.iterrows():
        file_name = row['file_name']
        
        # Si la fila no tiene prompt y existe en nuestro diccionario de prompts
        if (pd.isna(row['generated_prompt']) or row['generated_prompt'] is None or row['generated_prompt'] == '') and file_name in prompts_dict:
            df.at[idx, 'generated_prompt'] = prompts_dict[file_name]
            prompts_anadidos += 1
    
    # Guardar el CSV actualizado
    df.to_csv(guardar_como, index=False)
    print(f"Se añadieron {prompts_anadidos} prompts al CSV. Guardado como {guardar_como}")
    
    return df

def crear_csv_filas_sin_prompt(df, guardar_como):
    """
    Crea un nuevo CSV solamente con las filas que aún no tienen prompt.
    """
    # Filtrar las filas sin prompt (considerando valores nulos, None o cadenas vacías)
    df_sin_prompt = df[(df['generated_prompt'].isna()) | (df['generated_prompt'] == '') | (df['generated_prompt'].isnull())]
    
    # Guardar el nuevo CSV
    df_sin_prompt.to_csv(guardar_como, index=False)
    print(f"Se creó un CSV con {len(df_sin_prompt)} filas sin generated_prompt. Guardado como {guardar_como}")

def main():
    # Definir rutas de archivos
    archivo_csv = 'parte_4.csv'
    archivo_log = 'together4.log'
    csv_actualizado = 'parte_4_con_prompts.csv'
    csv_filas_sin_prompt = 'parte_4_faltantes.csv'
    
    # Comprobar que los archivos existen
    for archivo in [archivo_csv, archivo_log]:
        if not os.path.exists(archivo):
            print(f"Error: El archivo {archivo} no existe.")
            return
    
    # Extraer prompts del archivo de log
    prompts = extraer_prompts_del_log(archivo_log)
    
    # Actualizar el CSV con los prompts extraídos
    df_actualizado = actualizar_csv_con_prompts(archivo_csv, prompts, csv_actualizado)
    
    # Crear un CSV con las filas que aún no tienen prompt
    crear_csv_filas_sin_prompt(df_actualizado, csv_filas_sin_prompt)
    
    print("Proceso completado con éxito.")

if __name__ == "__main__":
    main()

Se encontraron 15101 prompts en el archivo de log.
Se ha añadido la columna 'generated_prompt' que no existía en el CSV original.
Se añadieron 15101 prompts al CSV. Guardado como parte_4_con_prompts.csv
Se creó un CSV con 5260 filas sin generated_prompt. Guardado como parte_4_faltantes.csv
Proceso completado con éxito.


Merge de dos archivos, de dos archivos, deben tener las mismas columnas

In [3]:
import pandas as pd
import os

def combinar_csvs(archivo_csv1, archivo_csv2, guardar_como):
    """
    Combina dos archivos CSV con la misma estructura en uno solo.
    Evita duplicados basados en la columna 'file_name'.
    """
    # Leer los dos CSV
    df1 = pd.read_csv(archivo_csv1)
    df2 = pd.read_csv(archivo_csv2)
    
    # Verificar que tienen las mismas columnas
    if list(df1.columns) != list(df2.columns):
        print("¡Advertencia! Los archivos CSV no tienen las mismas columnas:")
        print(f"Columnas en {archivo_csv1}: {list(df1.columns)}")
        print(f"Columnas en {archivo_csv2}: {list(df2.columns)}")
        return None
    
    # Combinar los dataframes
    df_combinado = pd.concat([df1, df2], ignore_index=True)
    
    # Eliminar duplicados basados en 'file_name' (conservando la primera ocurrencia)
    filas_antes = len(df_combinado)
    df_combinado.drop_duplicates(subset=['file_name'], keep='first', inplace=True)
    filas_eliminadas = filas_antes - len(df_combinado)
    
    # Guardar el CSV combinado
    df_combinado.to_csv(guardar_como, index=False)
    print(f"Se combinaron {len(df1)} filas del primer CSV y {len(df2)} filas del segundo CSV.")
    print(f"Se eliminaron {filas_eliminadas} filas duplicadas.")
    print(f"El CSV combinado tiene {len(df_combinado)} filas. Guardado como {guardar_como}")
    
    return df_combinado

def main():
    # Definir rutas de archivos por defecto
    archivo_csv1 = 'parte_3_faltantes.csv'
    archivo_csv2 = 'parte_4_faltantes.csv'
    csv_combinado = 'merge34_faltantes.csv'

    # Comprobar que los archivos existen
    for archivo in [archivo_csv1, archivo_csv2]:
        if not os.path.exists(archivo):
            print(f"Error: El archivo {archivo} no existe.")
            return
    
    # Combinar los CSVs
    combinar_csvs(archivo_csv1, archivo_csv2, csv_combinado)
    
    print("Proceso de combinación de CSVs completado con éxito.")

if __name__ == "__main__":
    main()

Se combinaron 4345 filas del primer CSV y 5260 filas del segundo CSV.
Se eliminaron 0 filas duplicadas.
El CSV combinado tiene 9605 filas. Guardado como merge34_faltantes.csv
Proceso de combinación de CSVs completado con éxito.
